# landscraper - doing the dirty work for intellectual property (IP) decisions

__Contributer: Akhil Jindal__ | https://github.com/akhil-jindal/

### Library Imports:

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
import re
from glob import glob

### Classes and functions:

In [ ]:
class Patent:
    
    def __init__(self, title, number, assignee, pub_date, source):
        """Defining pertinent attributes
        """
        self.title = title
        self.number = number
        self.assignee = assignee
        self.pub_date = pub_date
        self.source = source
           
    def soupify(self):
        """Soupify for BeautifulSoup
        """
        src = requests.get('{}'.format(self.source)).text
        soup = BeautifulSoup(src, 'lxml')
        return soup
    
    def description(self, soup):
        """Returns a detailed description of target patent application
        Can include Abstract, Summary and Background
        """
        description = soup.find('div', class_='description').text
        return description
      
    def genContent(self, classification, corpus_dir, soup):
        """Generates a training document for a corpus
        """
        class_dir = os.path.join(corpus_dir, classification)
        file_name = re.sub('[^A-Za-z0-9]+', '', self.number)
        
        if os.path.isdir(class_dir):
            write_path = os.path.join(class_dir, file_name)
            
            try:
                self.writeContent(write_path)
                
            except:
                 print ("Failed to generate {}".format(self.number))
        
        else:
            os.mkdir(class_dir)
            write_path = os.path.join(class_dir, file_name)
            
            try:
                self.writeContent(write_path)

            except:
                 print ("Failed to generate {}".format(self.number))
        
    
    def writeContent(self, write_file):
        """Writes the content for a training document
        """
        with open(write_file, "w") as wf:
            header = str(self.title) + "\n" + str(self.number) + "\n" + str(self.assignee) + "\n" + str(self.pub_date) + "\n" + str(self.source) + "\n"
            wf.write(header)
            try:
                desc = self.description(soup).encode('utf-8').strip()
                wf.write(desc)
            except:
                print ("Encoding messed up for {}".format(write_file))
        wf.close()

In [ ]:
def cleanCSV(csv_file):
    """The .csv file downloadable from patents.google.com.
    However, for our purposes, the downloaded .csv has more information than we need.
    We keep pertient columns, and delete rows that are associated with non-US patents.
    """
    
    df = pd.read_csv(csv_file, error_bad_lines=False, sep=',')
    df = df.drop(["inventor/author", "priority date", "filing/creation date", 
                  "grant date", "representative figure link"], axis = 1)
    df = df[df["id"].str.contains("US")]
    return df

def definePatent(dataframe):
    """Uses Patent class to generate patents for further processing
    """
    patents = []
    for index, row in dataframe.iterrows():
        patent = Patent(row["title"], row["id"], row["assignee"], row["publication date"], row["result link"])
        patents.append(patent)
    return patents

def classify(csv_file):
    classification = os.path.basename(csv_file).split(".")[0]
    return classification

### Preparing Corpus:

Note:

* The .csv's which contain identification information for each Patent were manually downloaded from patents.google.com.

* Certain certain classifications will yield more patents than others.  Down sampling may be necessary to ensure a balanced data set for training. 

__Inputs:__

In [ ]:
csv_dir = "/home/ajindal/Documents/landscraper/class_csv/"
corpus_dir = "/home/ajindal/Documents/landscraper/corpus"

__Getting Training Documents__

In [ ]:
globabble_dir = os.path.join(csv_dir, "*")

for csv_file in glob(globabble_dir):
    classification = classify(csv_file)
    
    print("Preparing the following class: {}".format(classification))
    clean = cleanCSV(csv_file)
   
    patents = definePatent(clean)
          
    for patent in patents:
        soup = patent.soupify()
        patent.genContent(classification, corpus_dir, soup)